<a href="https://colab.research.google.com/github/Mo92/Mo92/blob/main/obesity_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Aufgabenverteilung
Julia: Features 0–7 (Gender, Age, Height, Weight, family_history_with_overweight, FAVC, FCVC, NCP), Modell A (z.B. Logistic Regression)

Keisha: Features 8–15, Modell B (z.B. Random Forest)

Gruppe: Gemeinsame Kapitel (Business Understanding, Data Prep, Evaluation)

# 1. Business Understanding




1.1 Problemdefinition:

Die zunehmende Verbreitung von Übergewicht und Adipositas stellt weltweit eine ernstzunehmende Herausforderung für die öffentliche Gesundheit dar. Ungesunde Ernährungsgewohnheiten, Bewegungsmangel und ein sitzender Lebensstil sind bekannte Risikofaktoren, die zu erhöhten Gesundheitskosten und einer geringeren Lebensqualität führen. Ziel dieses Projekts ist es, ein datengetriebenes System zu entwickeln, das den Adipositas-Level einer Person anhand leicht zugänglicher Informationen über Ernährung und körperliche Aktivität automatisch klassifizieren kann.

1.2 Zielsetzung der Datenanalyse:

Mit Hilfe maschineller Lernverfahren sollen Muster und Zusammenhänge zwischen Lebensgewohnheiten und dem Adipositas-Level aufgedeckt werden. Anschließend wird ein Vorhersagemodell entwickelt, das die Adipositas-Klasse (z. B. Untergewicht, Normalgewicht, Übergewicht, Adipositas Typ I–III) zuverlässig bestimmen kann. Solch ein Modell könnte z. B. in digitalen Gesundheits-Apps oder im Rahmen präventiver Beratung eingesetzt werden.

1.3 Projekterfolgskriterien:

Ein generalisierbares Vorhersagemodell mit einer F1-Score ≥ 0.85 auf dem Testdatensatz
Interpretierbare Modellentscheidungen (z. B. mit SHAP)
Klar strukturierte und dokumentierte Ergebnisse
Verständliche Darstellung der Erkenntnisse, auch für nicht-technische Zielgruppen (z. B. Gesundheitsberater:innen)

1.4 Projektplan

| Aufgabe                       | Name      | Beschreibung                                      |
| ----------------------------- | --------- | ------------------------------------------------- |
| Business Understanding        | Gruppe    | Gemeinsame Zieldefinition und Planung             |
| EDA: Features 0–7             | \[Julia] | Analyse, Visualisierung und Bewertung             |
| EDA: Features 8–15            | \[Keisha] | Analyse, Visualisierung und Bewertung             |
| Datenvorbereitung             | Gruppe    | Gemeinsame Bearbeitung (z. B. Encoding, Cleaning) |
| Modell 1: Logistic Regression | \[Julia] | Eigene Pipeline, Optimierung, Evaluierung         |
| Modell 2: Random Forest       | \[Keisha] | Eigene Pipeline, Optimierung, Evaluierung         |
| Evaluation & Vergleich        | Gruppe    | Gemeinsame Bewertung und Entscheidung             |



# 2. Dataset


In [ ]:
# Bibliotheken
import pandas as pd
import plotly.express as px
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

In [ ]:
# fetch dataset
dataset= fetch_ucirepo(id=544)

# data (as pandas dataframes)
X = dataset.data.features
Y = dataset.data.targets

# metadata
print(dataset.metadata)

# variable information
print(dataset.variables)

# DataFrame
df = pd.concat([X, Y], axis=1)

#Zielspalte
target_col = 'NObeyesdad'


ConnectionError: Error connecting to server

# 3.1 EDA Features 0 - 7 Analyse (Julia)





Schritt 1: Setup und Datenauswahl

In [ ]:
# 1. Featureliste für Feature 0–7
features_0_7 = ['Gender', 'Age', 'Height', 'Weight',
                'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP']

print("=== EDA für Features 0–7 ===")



Schritt 2: Data Quality Check

In [ ]:
# === Data Quality Check (Julia) ===

# 1. Fehlende Werte prüfen
print("Fehlende Werte pro Feature:")
print(df[features_0_7].isnull().sum())

# 2. Datentypen prüfen
print("\nDatentypen der Features:")
print(df[features_0_7].dtypes)

# 3. Duplikate prüfen (nur Features 0–7)
duplicates = df[features_0_7].duplicated(keep=False)  # Alle Duplikate markieren (inkl. erster Vorkommen)
print(f"\nGesamtzahl der Duplikate (alle betroffenen Zeilen): {duplicates.sum()}")

# 4. Duplikatgruppen analysieren (inkl. Target)
duplicate_groups = df[features_0_7 + ['NObeyesdad']].groupby(features_0_7).filter(lambda x: len(x) > 1)
print(f"\nAnzahl der Duplikatgruppen: {duplicate_groups.groupby(features_0_7).ngroups}")
print(f"Gesamtzahl der Zeilen in Duplikatgruppen: {len(duplicate_groups)}")

# 5. Duplikatgruppen anzeigen
print("\nAlle Duplikatgruppen (Features 0–7 + Target):")
if not duplicate_groups.empty:
    display(duplicate_groups.sort_values(by=features_0_7))
else:
    print("Keine Duplikate gefunden.")

# 6. Konfliktanalyse (unterschiedliches Target in gleichen Features)
conflicting_targets = duplicate_groups.groupby(features_0_7)['NObeyesdad'].nunique() > 1
print(f"\nFälle mit gleichen Features, aber unterschiedlichem Target: {conflicting_targets.sum()}")



Data Quality Check Notes(Julia)

- **Fehlende Werte:** Keine in Features 0–7.
- **Datentypen:** Korrekt erkannt (numerisch, kategorial, binär).
- **Duplikate:**  
  - 25 Duplikatgruppen  
  - 35 betroffene Zeilen  
  - Keine Konflikte im Target  
- **Notes:**  
  Die identifizierten echten Duplikate (Features + Target identisch) **werden bei Data Preparation entfernt**.

```




Schritt 3: Deskriptive Statistik & Verteilungsanalyse pro Feature:

| Feature | Typ         | Beschreibung                                 | Wertebereich/Kodierung           |
|---------|-------------|----------------------------------------------|----------------------------------|
| Gender  | Kategorial  | Geschlecht                                   | Male, Female                     |
| Age     | Numerisch   | Alter in Jahren                              | 14–61                            |
| Height  | Numerisch   | Körpergröße in Meter                         | 1.45–1.98                        |
| Weight  | Numerisch   | Gewicht in kg                                | 39–173                           |
| family_history_with_overweight | Binär | Familiäre Vorbelastung mit Übergewicht | yes, no           |
| FAVC    | Binär       | Häufiger Konsum hochkalorischer Nahrung      | yes, no                          |
| FCVC    | Ordinal     | Gemüseverzehr (1=selten, 3=oft)              | 1.0–3.0                          |
| NCP     | Numerisch   | Hauptmahlzeiten pro Tag                      | 1.0–4.0                          |


Feature Gender

In [ ]:
feature = 'Gender'

# Absolute und prozentuale Häufigkeiten berechnen
gender_counts = df[feature].value_counts()
gender_props = df[feature].value_counts(normalize=True) * 100

print(f"=== Feature: {feature} ===")
print(gender_counts)
print(gender_props.round(1))

Visualisierung

In [ ]:
import plotly.express as px
gender_df = pd.DataFrame({
    "Gender": gender_counts.index,
    "Anzahl": gender_counts.values,
    "Prozent": gender_props.values
})

fig = px.bar(
    gender_df,
    x="Gender",
    y="Anzahl",
    color="Gender",
    text=gender_df["Prozent"].apply(lambda x: f"{x:.1f}%"),
    title="Verteilung der Geschlechter im Datensatz",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_traces(textposition='outside')
fig.update_layout(showlegend=False, yaxis_title="Anzahl", xaxis_title="Geschlecht")
fig.show()


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Absolute Häufigkeiten
cross_tab = pd.crosstab(df['Gender'], df['NObeyesdad'])
print("Absolute Häufigkeiten:")
display(cross_tab)

# Chi-Quadrat-Test auf Unabhängigkeit
chi2, p, dof, expected = chi2_contingency(cross_tab)
print(f"\nChi-Quadrat-Test: p-Wert = {p:.4f}")

# Interpretation
if p < 0.05:
    print("Signifikanter Zusammenhang zwischen Gender und Adipositas-Level (p < 0.05).")
else:
    print("Kein signifikanter Zusammenhang (p ≥ 0.05).")

In [ ]:
import plotly.express as px

fig = px.bar(
    cross_tab.reset_index(),
    x='Gender',
    y=cross_tab.columns.tolist(),
    title="Adipositas-Level nach Geschlecht",
    labels={'value': 'Anzahl', 'Gender': 'Geschlecht'},
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(barmode='group')  # 'group' für nebeneinander, 'stack' für gestapelt
fig.show()


Zusammenhang zwischen Gender und Adipositas-Level (Julia)
Analyse:
Es wurde eine Kreuztabelle zwischen Gender und dem Target (NObeyesdad) erstellt. Die absoluten Häufigkeiten zeigen deutliche Unterschiede in den Adipositas-Leveln zwischen Männern und Frauen.

Statistik:
Ein Chi-Quadrat-Test auf Unabhängigkeit ergab einen hochsignifikanten Zusammenhang zwischen Gender und Adipositas-Level (p = 0.0000, also p < 0.05).

Visualisierung:
Das gruppierte Balkendiagramm zeigt, dass:

Frauen häufiger in den Kategorien Insufficient_Weight, Obesity_Type_III und Overweight_Level_I vertreten sind.

Männer häufiger in den Kategorien Obesity_Type_II, Normal_Weight und Overweight_Level_II vorkommen.

Interpretation:
Es gibt einen statistisch signifikanten Zusammenhang zwischen Geschlecht und Adipositas-Level. Die Verteilung der Adipositas-Kategorien unterscheidet sich deutlich zwischen Männern und Frauen.
Dies stimmt mit der Literatur überein, die geschlechtsspezifische Unterschiede im Risiko und Verlauf von Übergewicht und Adipositas beschreibt (z.B. hormonelle, genetische und soziokulturelle Faktoren).

Relevanz:
Das Feature Gender sollte im Modell berücksichtigt werden, da es einen Einfluss auf die Zielvariable hat.



Feature Age

In [ ]:
feature = 'Age'
print(df[feature].describe())
print(f"Median: {df[feature].median():.1f}")
print(f"IQR: {df[feature].quantile(0.75) - df[feature].quantile(0.25):.1f}")

In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x='Age',
    nbins=20,
    marginal='box',  # Zeigt Boxplot oberhalb des Histogramms
    title='Altersverteilung der Stichprobe',
    labels={'Age': 'Alter (Jahre)', 'count': 'Anzahl'},
    color_discrete_sequence=['#8ecae6']
)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x='Age',
    facet_col='NObeyesdad',
    color='NObeyesdad',
    title='Altersverteilung je Obesity-Level',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    nbins=20
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
print(df.groupby('NObeyesdad')['Age'].describe())

In [ ]:
numerical_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP']
corr = df[numerical_features].corr()
fig = px.imshow(corr, text_auto=True, title='Korrelationsmatrix')
fig.show()

**Was sagen die Werte in deinem Plot?**

Age und Height:
Korrelation: -0.03
→ Kein Zusammenhang zwischen Alter und Körpergröße.

Height und Weight:
Korrelation: 0.46
→ Mittlere positive Korrelation: Größere Personen wiegen tendenziell mehr.

Weight und FCVC:
Korrelation: 0.22
→ Schwacher positiver Zusammenhang: Höheres Gewicht ist leicht mit häufigerem Gemüseverzehr assoziiert (kann Zufall sein).

FCVC und NCP:
Korrelation: 0.03
→ Praktisch kein Zusammenhang zwischen Gemüseverzehr und Anzahl der Hauptmahlzeiten.

Alle anderen Werte:
Liegen nahe bei 0 → Kein linearer Zusammenhang.
Was bedeutet das für deine Analyse?
Starke Korrelationen (>0.7 oder <-0.7): Keine vorhanden.
→ Keine Gefahr von starker Multikollinearität.

Mittlere Korrelation:
Nur zwischen Height und Weight (0.46) – das ist physiologisch plausibel.

Fast alle anderen Features sind unabhängig voneinander.

In [ ]:
import numpy as np

def correlation_ratio(categories, values):
    # categories: Target (z.B. NObeyesdad)
    # values: numerisches Feature (z.B. Age)
    categories = np.array(categories)
    values = np.array(values)
    category_means = [values[categories == cat].mean() for cat in np.unique(categories)]
    n = np.array([sum(categories == cat) for cat in np.unique(categories)])
    grand_mean = values.mean()
    numerator = sum(n * (category_means - grand_mean) ** 2)
    denominator = sum((values - grand_mean) ** 2)
    eta = np.sqrt(numerator / denominator)
    return eta

for feature in numerical_features:
    eta = correlation_ratio(df['NObeyesdad'], df[feature])
    print(f"Korrelationsverhältnis (Eta) für {feature} und NObeyesdad: {eta:.2f}")

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Funktion für Korrelationsverhältnis (Eta)
def correlation_ratio(categories, values):
    categories = np.array(categories)
    values = np.array(values)
    cats = np.unique(categories)
    n = len(values)
    grand_mean = values.mean()
    numerator = sum([(values[categories == cat].mean() - grand_mean)**2 * sum(categories == cat) for cat in cats])
    denominator = sum((values - grand_mean)**2)
    return np.sqrt(numerator / denominator) if denominator != 0 else 0

# Berechne Eta für alle numerischen Features vs. Target
eta_values = {}
for feature in numerical_features:
    eta = correlation_ratio(df['NObeyesdad'], df[feature])
    eta_values[feature] = eta

# Erstelle DataFrame für die Matrix
eta_df = pd.DataFrame({
    'Feature': numerical_features,
    'NObeyesdad (Eta)': [eta_values[feature] for feature in numerical_features]
}).set_index('Feature')

# Visualisierung mit Heatmap
fig = px.imshow(
    eta_df.T,  # Transponiert, um Features in Zeilen und Target in Spalten zu zeigen
    text_auto=".2f",
    color_continuous_scale='Blues',
    title="Korrelationsverhältnis (Eta) zwischen numerischen Features und Target"
)
fig.update_layout(xaxis_title="Feature", yaxis_title="Target (NObeyesdad)")
fig.show()

#### Korrelationsverhältnis (Eta) zwischen numerischen Features und Target (Julia)

- **Weight** (Eta = 0.92): Sehr starker Zusammenhang mit dem Adipositas-Level – das Feature ist für die Vorhersage entscheidend.
- **FCVC** (Eta = 0.49) und **Age** (Eta = 0.43): Mittlerer Zusammenhang, beide Features tragen relevant zur Modellierung bei.
- **Height** (Eta = 0.31) und **NCP** (Eta = 0.27): Schwacher Zusammenhang, geringere Bedeutung für die Zielvariable.

**Fazit:**  
Weight ist das wichtigste numerische Feature für die Vorhersage von NObeyesdad, gefolgt von FCVC und Age. Height und NCP sind weniger relevant, können aber als Zusatzinformation im Modell verbleiben.

**Visualisierung:**  
Die Heatmap zeigt die Stärke des Zusammenhangs farbcodiert und numerisch an.  


Feature Weight


In [ ]:
feature = 'Weight'

# Deskriptive Statistik
weight_desc = df[feature].describe()
weight_median = df[feature].median()
weight_iqr = df[feature].quantile(0.75) - df[feature].quantile(0.25)

print(f"=== Feature: {feature} ===")
print(weight_desc)
print(f"Median: {weight_median:.1f}")
print(f"IQR: {weight_iqr:.1f}")

In [ ]:
import plotly.express as px

fig = px.box(
    df,
    x='NObeyesdad',
    y='Weight',
    color='NObeyesdad',
    title='Gewichtsverteilung pro Adipositas-Level',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(
    xaxis_title="Obesity-Level",
    yaxis_title="Gewicht (kg)",
    showlegend=False
)
fig.show()



In [ ]:
fig = px.histogram(
    df,
    x='Weight',
    facet_col='NObeyesdad',
    color='NObeyesdad',
    title='Gewichtsverteilung je Adipositas-Level',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    nbins=20
)
fig.update_layout(showlegend=False)
fig.show()


Feature: hight


In [ ]:
height_desc = df['Height'].describe()
height_median = df['Height'].median()
height_iqr = df['Height'].quantile(0.75) - df['Height'].quantile(0.25)

print(f"=== Feature: Height ===")
print(height_desc)
print(f"Median: {height_median:.2f}")
print(f"IQR: {height_iqr:.2f}")


In [ ]:
fig = px.violin(
    df,
    x='NObeyesdad',
    y='Weight',
    color='NObeyesdad',
    box=True,
    points="all",
    title='Gewichtsverteilung pro Adipositas-Level (Violinplot)',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(
    xaxis_title='Adipositas-Level',
    yaxis_title='Gewicht (kg)',
    showlegend=False
)
fig.show()



In [ ]:
df['BMI'] = df['Weight'] / (df['Height'] ** 2)

fig.show()

In [ ]:
# Beispielplots mit Zielvariable
for feature in features_0_7:
    if df[feature].dtype == 'object' or df[feature].nunique() < 10:
        fig = px.histogram(df, x=feature, color=target_col, barmode="group",
                           title=f"{feature} nach Zielklasse")
    else:
        fig = px.box(df, x=target_col, y=feature, title=f"{feature} nach Zielklasse")
    fig.show()

Data Quality Check:

# 3.2 EDA Features 9-16

In [ ]:
# Featureliste für Feature 8–15
features_8_15 = ['CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS']

print("=== EDA für Features 8–15 ===")

# Fehlende Werte
print("\nFehlende Werte:")
print(df[features_8_15].isnull().sum())

# Datentypen
print("\nDatentypen:")
print(df[features_8_15].dtypes)





Was bedeutet das alles?


In [ ]:
print("\n=== PRÜFUNG AUF DOPPELTE ZEILEN ===")

# Nur in diesen Spalten nach Duplikaten suchen
duplicates_8_15 = df[features_8_15].duplicated()
num_duplicates = duplicates_8_15.sum()

print(f"Anzahl doppelter Zeilen (Features 8–15): {num_duplicates}")

# Optional: Doppelte Zeilen anzeigen
if num_duplicates > 0:
    print("\nFolgende Zeilen sind Duplikate:")
    print(df[features_8_15][duplicates_8_15])
else:
    print("\nKeine Duplikate gefunden.")

In [ ]:
# Beispielplots mit Zielvariable
for feature in features_8_15:
    if df[feature].dtype == 'object' or df[feature].nunique() < 10:
        fig = px.histogram(df, x=feature, color=target_col, barmode="group",
                           title=f"{feature} nach Zielklasse")
    else:
        fig = px.box(df, x=target_col, y=feature, title=f"{feature} nach Zielklasse")
    fig.show()

# 3.3 EDA Target

In [ ]:
print("\n=== Zielvariable: NObeyesdad ===")

# Klassenverteilung anzeigen
print(df[target_col].value_counts())

# Visualisierung
fig = px.histogram(df, x=target_col, title="Verteilung der Zielklassen")
fig.show()

# 4. Data Preparation

In [ ]:
df_cleaned = df[~(df[features_8_15].eq(df[features_8_15].shift(1)).all(axis=1) &
                  df[features_8_15].eq(df[features_8_15].shift(2)).all(axis=1))].reset_index(drop=True)

print("Anomalien")

# 5. Modell 1: Logistic Regression

# 6. Modell 2: Random Forest

# 7. Evaluation und Vergleich